In [1]:
import kerchunk
import xarray as xr
from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr
import fsspec
from glob import glob
import xradar as xd
import matplotlib.pyplot as plt
import cmweather
import numpy as np
import hvplot.xarray
import cartopy.crs as ccrs
import pyproj
from wradlib.georef import epsg_to_osr, georeference
from xarray.core.datatree import DataTree
from zarr.errors import ContainsGroupError  

In [3]:
def fix_angle(ds: xr.Dataset, exp_dim="volume_time", tolerance=1.0, **kwargs) -> xr.Dataset:
    """
    This function reindex the radar azimuth angle to make all sweeps starts and end at the same angle
    @param ds: xarray dataset containing and xradar object
    @return: azimuth reindex xarray dataset
    """
    ds["time"] = ds.time.load() 
    angle_dict = xd.util.extract_angle_parameters(ds)
    start_ang = angle_dict["start_angle"]
    stop_ang = angle_dict["stop_angle"]
    direction = angle_dict["direction"]
    ds = xd.util.remove_duplicate_rays(ds)
    az = len(np.arange(start_ang, stop_ang))
    ar = np.round(az / len(ds.azimuth.data), 2)
    ds = xd.util.reindex_angle(
        ds, start_ang, stop_ang, ar, direction, method="nearest", tolerance=tolerance, **kwargs
    )
        
    ds = ds.expand_dims(exp_dim)  # Expand for volumes for concatenation
    ds[exp_dim] = [np.nanmin(ds.time.values)]
    
    return ds

In [2]:
radars = {
    "cband": "../radar/erad2024/20240522_MeteoSwiss_ARPA_Lombardia/Data/Cband/", 
    "xband": "../radar/erad2024/20240522_MeteoSwiss_ARPA_Lombardia/Data/Xband/",
         }

In [13]:
for key in list(radars.keys()):
    print(key)
    erad_files = sorted(glob(f"{radars[key]}*.nc"))
    dt = xd.io.open_cfradial1_datatree(erad_files[0])
    groups = [
        i[1:] for i in list(dt.groups) if i.startswith("/sweep") if i not in ["/"]
    ]
    for group in groups:
        root = {}
        ds = xr.open_mfdataset(
            erad_files,
            preprocess=fix_angle,
            engine="cfradial1",
            group=group,
            concat_dim="volume_time",
            combine="nested",
        ).xradar.georeference()
        root[f"{key}/{group}"] = ds
        dtree = DataTree.from_dict(root)
        try:
            dtree.to_zarr("../radar/erad2024/zarr_radar/erad_2024.zarr", consolidated=True)
        except ContainsGroupError:
            dtree.to_zarr("../radar/erad2024/zarr_radar/erad_2024.zarr", consolidated=True, mode="a")
    
        del dtree, ds

cband
xband


In [10]:
dt = xr.backends.api.open_datatree("https://js2.jetstream-cloud.org:8001/pythia/radar/erad2024/zarr_radar/erad_2024.zarr", engine="zarr", chunks={})

In [11]:
dt

<xarray.DataTree>
Group: /

In [5]:
# Set the URL and path for the cloud
URL = 'https://js2.jetstream-cloud.org:8001/'
path = f'pythia/radar/erad2024'

fs = fsspec.filesystem("s3", anon=True, client_kwargs=dict(endpoint_url=URL))

fs.glob(f"{path}/*")

['pythia/radar/erad2024/20240522_MeteoSwiss_ARPA_Lombardia',
 'pythia/radar/erad2024/dda_data']

In [6]:
mapper = fs.get_mapper(f"{path}/erad_2024.zarr")

In [7]:
%%time
dtree.to_zarr(mapper, consolidated=True, mode="w")

KeyError: '.zgroup'